### Задание
Реализовать суммаризацию текста
Взять тот же датасет, который был на вебинаре и предобученную модель для задачи суммаризации
1. Проверить насколько хорошо она суммаризирует
2. (дополнительно) Сделать генерацию заголовков для статьи (обучить модель для генерации заголовков)

In [1]:
!pip install transformers sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 51.2 MB/s eta 0:00:00


In [2]:
!pip3 install torch torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!wget -q https://www.dropbox.com/s/43l702z5a5i2w8j/gazeta_train.txt 
!wget -q https://www.dropbox.com/s/k2egt3sug0hb185/gazeta_val.txt 
!wget -q https://www.dropbox.com/s/3gki5n5djs9w0v6/gazeta_test.txt

In [4]:
import json
import random
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [5]:
def read_gazeta_records(file_name, shuffle=True, sort_by_date=False):
    assert shuffle != sort_by_date
    records = []
    with open(file_name, "r") as r:
        for line in r:
            records.append(json.loads(line))
    if sort_by_date:
        records.sort(key=lambda x: x["date"])
    if shuffle:
        random.shuffle
    return records

def summurize(rec):
    article_text = ("{}".format(test_records[rec]["text"]))
    input_ids = tokenizer([HANDLER(article_text)], 
                          return_tensors="pt", 
                          padding="max_length", 
                          truncation=True, 
                          max_length=512)["input_ids"]
    output_ids = model.generate(input_ids=input_ids,
                                max_length=84,
                                no_repeat_ngram_size=2,
                                num_beams=4)[0]
    summary = tokenizer.decode(output_ids,skip_special_tokens=True,clean_up_tokenization_spaces=False)
    print('TEXT:   ', article_text)
    print('SUMMARY:', summary)

In [6]:
test_records = read_gazeta_records("gazeta_test.txt")

In [7]:
HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))
model_name = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.8/dist-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [8]:
summurize(1)

TEXT:    Около 11 тысяч зрителей увидели все самое лучшее, что есть на сегодняшний день в культуре Бурятии. В Кремле выступил Бурятский государственный академический театр оперы и балета, Национальный цирк, Бурятский национальный театр песни и танца «Байкал», ставший победителем шоу «Танцуют все!» на телеканале «Россия», а также другие профессиональные и самодеятельные коллективы региона. Более 300 артистов из одного региона на главной сцене страны - похоже это рекорд России. Зрителям рассказали, что Республике Бурятия, чье население составляет 1 миллион человек, сохранилась и развивается культура десятков национальностей, включая русских, бурятов, староверов (семейских), эвенков. И все они были представлены в Москве. Как писали после шоу зрители в соцсетях: «А мы думали, что в Бурятии только буряты живут…». Для неподготовленного зрителя это вообще были вечера открытий. Например, когда еще в Кремлевском дворце выстраивалась очередь из желающих попасть на прием к врачам-пульсологам и ла

In [9]:
summurize(7)

TEXT:    Самые известные российские пранкеры Владимир «Вован» Кузнецов и Алексей «Лексус» Столяров, на счету которых множество розыгрышей мировых политиков первого эшелона, позвонили члену конгресса США от Демократической партии Максин Уотерс от имени шведской школьницы, экологической активистки Греты Тунберг и ее отца Сванте. Звонок был сделан в рамках нового анимационного шоу пранкеров Stars save the Earth. Россияне рассказали американскому политику, что Грета якобы сейчас участвует в митинге в штате Северная Каролина «в поддержку экологии острова Чунга-Чанга». В этой связи «Тунберги» предложили Уотерс также поддержать экологию острова (несуществующего), чтобы ее речь затем якобы транслировать на мероприятии, передает РИА «Новости». «Я очень рада, что Грета с вами в Северной Каролине, где вы сосредоточены на защите очень важного острова Чунга-Чанга. Самое главное сейчас — убедиться, что остров защищен, что вы даете ему всю поддержку, которую вы можете дать», — заявила конгрессвумен С

In [10]:
summurize(14)

TEXT:    Звезда сериала «Как я встретил вашу маму» Нил Патрик Харрис присоединился к актерскому составу «Матрицы 4», передает Variety. О том, кого именно в грядущем фильме Ланы Вачовски сыграет 46-летний американец, на данный момент не сообщается. Ранее в каст проекта вошли уже легендарные для франшизы Киану Ривз и Кэри-Энн Мосс (им предстоит вновь исполнить роли Нео и Тринити соответственно), а также Яхья Абдул-Матин II («Черное зеркало», «Рассказ служанки», «Аквамен», «Мы»), который, по информации СМИ, сыграет в картине одного из главных персонажей. Сценарием грядущего фантастического фильма — помимо Вачовски, создательницы оригинальной трилогии — занимаются писатель Александр Хемон и автор «Облачного атласа» Дэвид Митчелл. Продюсирует ленту Грант Хилл («Титаник»), давний знакомый Ланы по все той же «Матрице». Операторское кресло досталось Джону Толлу, двукратному лауреату премии «Оскар» («Храброе сердце», «Легенды осени»). Съемки ленты, как ожидается, начнутся 10 февраля 2020 года н

In [11]:
summurize(21)

TEXT:    Украинские элиты стремятся стать максимально независимыми от Киева, потому что при продолжении нынешней политики президента страны Владимира Зеленского Украина распадется на пять государств, заявил грузинский политик Михаил Саакашвили. По его словам, ряд высокопоставленных украинских лиц имеют свои частные армии, а также внушительные финансовые ресурсы, чтобы в случае какого-либо политического или социального кризиса на Украине отделиться от Киева, передает ФАН. «Вы должны понять, что эти местные люди, которые уже обросли деньгами, частными армиями… К примеру, у [мэра Одессы] Геннадия Труханова есть частная армия, у харьковского мэра Геннадия Кернеса есть своя армия… Они готовят инфраструктуру для того, чтобы Украина превратилась в еще пять Украин», — пояснил бывший президент Грузии. Он подчеркнул, что страну ждет экономическая деградация и последующий развал. Зеленский и вся его команда отдали страну на аутсорсинг – под внешнее управление крупных глобальных финансово-промышле